In [589]:
import pandapower.networks as pn
import pandapower.converter as pc

In [590]:
net = pn.case118()

In [591]:
ppc = pc.to_ppc(net, init='flat')

In [592]:
from pandapower.pypower.makePTDF import makePTDF
ptdf = makePTDF(ppc['baseMVA'], ppc['bus'], ppc['branch'])

In [593]:
ptdf

array([[ 3.82812860e-01, -2.58527225e-01,  2.30588833e-01, ...,
         8.34429759e-05, -3.73715621e-02, -6.58885699e-05],
       [ 6.17187140e-01,  2.58527225e-01, -2.30588833e-01, ...,
        -8.34429759e-05,  3.73715621e-02,  6.58885699e-05],
       [ 3.79052401e-02,  8.89963326e-02,  1.62209325e-02, ...,
        -3.63047261e-04,  1.20499949e-01,  2.63119602e-04],
       ...,
       [ 8.71120163e-02,  8.70171758e-02,  8.71522689e-02, ...,
         1.65496795e-01,  8.69586956e-02, -1.87806120e-01],
       [ 2.34102394e-17,  1.90241995e-17,  2.04086353e-17, ...,
         3.32925994e-19,  1.63196944e-17, -4.69052869e-19],
       [-1.56980346e-14, -1.26965151e-14, -1.36431551e-14, ...,
         1.00000000e+00, -1.08457283e-14, -3.75242295e-16]])

In [594]:
ppc['branch']

array([[0.00000000e+00, 1.00000000e+00, 3.03000000e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.00000000e+00, 1.29000000e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.00000000e+00, 4.00000000e+00, 1.76000000e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [8.00000000e+01, 7.90000000e+01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.60000000e+01, 8.50000000e+01, 2.81494977e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.15000000e+02, 6.70000000e+01, 3.39887086e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [595]:
from pandapower.pypower.makeLODF import makeLODF
lodf = makeLODF(ppc['branch'], ptdf)

D:\Programs\anaconda3\envs\gurobi\lib\site-packages\pandapower\pypower\makeLODF.py:28: RuntimeWarning: invalid value encountered in double_scalars
  LODF[ix, ix] -= (LODF[ix, ix] + 1.)


In [596]:
T = 24
Nb = ppc['bus'].shape[0]
Nl = ppc['branch'].shape[0]
Ng = ppc['gen'].shape[0]

In [597]:
Nb, Nl, Ng

(118, 186, 54)

In [598]:
load_percent_daily = [74, 70, 66, 65, 64, 62, 62, 66, 81, 86, 91, 93, 93, 92, 91, 91, 92, 94, 95, 95, 100, 93, 88, 80]
# load_percent_daily = [34, 20, 36, 45, 44, 52, 52, 66, 71, 76, 91, 93, 103, 82, 71, 72, 52, 44, 45, 55, 40, 33, 28, 10]
len(load_percent_daily)

24

In [599]:
load_percent_daily = [load_percent / 100 for load_percent in load_percent_daily]

In [600]:
load_percent_daily

[0.74,
 0.7,
 0.66,
 0.65,
 0.64,
 0.62,
 0.62,
 0.66,
 0.81,
 0.86,
 0.91,
 0.93,
 0.93,
 0.92,
 0.91,
 0.91,
 0.92,
 0.94,
 0.95,
 0.95,
 1.0,
 0.93,
 0.88,
 0.8]

In [601]:
from pandapower.pypower.idx_bus import PD
import numpy as np
demand = ppc['bus'][:, PD].reshape((-1, 1)) @ np.array([load_percent_daily]).reshape((1, -1))

In [602]:
demand

array([[ 37.74,  35.7 ,  33.66, ...,  47.43,  44.88,  40.8 ],
       [ 14.8 ,  14.  ,  13.2 , ...,  18.6 ,  17.6 ,  16.  ],
       [ 28.86,  27.3 ,  25.74, ...,  36.27,  34.32,  31.2 ],
       ...,
       [136.16, 128.8 , 121.44, ..., 171.12, 161.92, 147.2 ],
       [ 14.8 ,  14.  ,  13.2 , ...,  18.6 ,  17.6 ,  16.  ],
       [ 24.42,  23.1 ,  21.78, ...,  30.69,  29.04,  26.4 ]])

In [603]:
demand.shape == (Nb, T)

True

In [604]:
from pandapower.pypower.idx_cost import STARTUP, SHUTDOWN, NCOST
from pandapower.pypower.idx_gen import PMIN, PMAX
n_sample = 15
p_sample = np.array([ppc['gen'][:, PMIN] + s * (ppc['gen'][:, PMAX] - ppc['gen'][:, PMIN]) / n_sample for s in range(n_sample + 1)]).T

In [605]:
p_sample

array([[-1.00000000e-10,  5.36800000e+01,  1.07360000e+02,
         1.61040000e+02,  2.14720000e+02,  2.68400000e+02,
         3.22080000e+02,  3.75760000e+02,  4.29440000e+02,
         4.83120000e+02,  5.36800000e+02,  5.90480000e+02,
         6.44160000e+02,  6.97840000e+02,  7.51520000e+02,
         8.05200000e+02],
       [-1.00000000e-10,  6.66666667e+00,  1.33333333e+01,
         2.00000000e+01,  2.66666667e+01,  3.33333333e+01,
         4.00000000e+01,  4.66666667e+01,  5.33333333e+01,
         6.00000000e+01,  6.66666667e+01,  7.33333333e+01,
         8.00000000e+01,  8.66666667e+01,  9.33333333e+01,
         1.00000000e+02],
       [-1.00000000e-10,  6.66666667e+00,  1.33333333e+01,
         2.00000000e+01,  2.66666667e+01,  3.33333333e+01,
         4.00000000e+01,  4.66666667e+01,  5.33333333e+01,
         6.00000000e+01,  6.66666667e+01,  7.33333333e+01,
         8.00000000e+01,  8.66666667e+01,  9.33333333e+01,
         1.00000000e+02],
       [-1.00000000e-10,  6.66666667e

In [606]:
ppc['gen'][:, [PMIN, PMAX]]

array([[-1.000e-10,  8.052e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  5.500e+02],
       [-1.000e-10,  1.850e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  3.200e+02],
       [-1.000e-10,  4.140e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.070e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.190e+02],
       [-1.000e-10,  3.040e+02],
       [-1.000e-10,  1.480e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  2.550e+02],
       [-1.000e-10,  2.600e+02],
       [-1.000e-10,  1.000e+02],
       [-1.000e-10,  4.910e+02],
       [-1.000e-10,  4.920e+02],
       [-1

In [607]:
ppc['gencost'][:10]


array([[2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.93648e-02,
        2.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 2.22222e-02,
        2.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.17647e-01,
        2.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000

In [608]:
cost_sample = np.zeros((Ng, n_sample + 1))
from pandapower.pypower.idx_cost import COST
for k in range(Ng):
    p = np.ones(n_sample + 1)
    for i in range(int(ppc['gencost'][k, NCOST])):
        cost_sample[k, :] += p * ppc['gencost'][k, COST + NCOST - i - 1]
        p *= p_sample[k, :]
cost_sample

array([[-2.00000000e-09,  1.12940049e+03,  2.37040197e+03,
         3.72300443e+03,  5.18720788e+03,  6.76301231e+03,
         8.45041772e+03,  1.02494241e+04,  1.21600315e+04,
         1.41822399e+04,  1.63160492e+04,  1.85614596e+04,
         2.09184709e+04,  2.33870832e+04,  2.59672965e+04,
         2.86591108e+04],
       [-4.00000000e-09,  2.67111111e+02,  5.35111111e+02,
         8.04000000e+02,  1.07377778e+03,  1.34444444e+03,
         1.61600000e+03,  1.88844444e+03,  2.16177778e+03,
         2.43600000e+03,  2.71111111e+03,  2.98711111e+03,
         3.26400000e+03,  3.54177778e+03,  3.82044444e+03,
         4.10000000e+03],
       [-4.00000000e-09,  2.67111111e+02,  5.35111111e+02,
         8.04000000e+02,  1.07377778e+03,  1.34444444e+03,
         1.61600000e+03,  1.88844444e+03,  2.16177778e+03,
         2.43600000e+03,  2.71111111e+03,  2.98711111e+03,
         3.26400000e+03,  3.54177778e+03,  3.82044444e+03,
         4.10000000e+03],
       [-4.00000000e-09,  2.67111111e

In [609]:
ppc['gencost']

array([[2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.93648e-02,
        2.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 2.22222e-02,
        2.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.17647e-01,
        2.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e+00, 1.00000e-02,
        4.00000e+01, 0.00000e+00],
       [2.00000e+00, 0.00000

In [610]:
su = ppc['gencost'][:, STARTUP]
sd = ppc['gencost'][:, SHUTDOWN]

In [611]:
print(demand.shape, su.shape, sd.shape, p_sample.shape, cost_sample.shape, ptdf.shape)

(118, 24) (54,) (54,) (54, 16) (54, 16) (186, 118)


In [612]:
ru = (ppc['gen'][:, PMIN] + ppc['gen'][:, PMAX]) / 4
tu = np.random.randint(1, 3, Ng)
td = np.random.randint(1, 3, Ng)

In [613]:
print(ru.shape, tu.shape, td.shape)

(54,) (54,) (54,)


In [614]:
from pandapower.pypower.idx_brch import RATE_A
pl_max = ppc['branch'][:, RATE_A]

In [615]:
print(pl_max.shape)

(186,)


In [616]:
import gurobipy as gp
from gurobipy import GRB
m = gp.Model()
# unit status
u = m.addVars(Ng, T, vtype=GRB.BINARY, name='u')
y = m.addVars(Ng, T, vtype=GRB.BINARY, name='y')
z = m.addVars(Ng, T, vtype=GRB.BINARY, name='z')
# unit power
p = m.addVars(Ng, T, name='p', lb=0)
alpha = m.addVars(n_sample + 1, Ng, T, name='alpha', lb=0.0)    # alpha >= 0

In [617]:
Demand = gp.tupledict({(b, t): demand[b][t] for b in range(Nb) for t in range(T)})
P_sample = gp.tupledict({(k, s): p_sample[k][s] for k in range(Ng) for s in range(n_sample + 1)})
Cost_sample = gp.tupledict({(k, s): cost_sample[k][s] for k in range(Ng) for s in range(n_sample + 1)})
_, SU, SD, RU, RD, TU, TD = gp.multidict({k: [su[k], sd[k], ru[k], -ru[k], tu[k], td[k]] for k in range(Ng)})

In [618]:
m.printStats()


Statistics for modelUnnamed:
  Linear constraint matrix    : 0 Constrs, 0 Vars, 0 NZs
  Matrix coefficient range    : [ 0, 0 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 0, 0 ]


In [619]:
f = m.addVars(Ng, T, name='f')

In [620]:
m.addConstrs((f[k, t] == gp.quicksum([alpha[s, k, t] * Cost_sample[k, s] for s in range(n_sample + 1)]) for k in range(Ng) for t in range(T)), name='cost_sample')
m.addConstrs((p[k, t] == gp.quicksum([alpha[s, k, t] * P_sample[k, s] for s in range(n_sample + 1)]) for k in range(Ng) for t in range(T)), name='p_sample')
m.addConstrs((alpha.sum('*', k, t) == u[k, t] for k in range(Ng) for t in range(T)), name='alpha_sum')      # important!

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [621]:
m.setObjective(gp.quicksum((f[k, t] + SU[k] * y[k, t] + SD[k] * z[k, t] for k in range(Ng) for t in range(T))))

In [622]:
m.printStats()


Statistics for modelUnnamed:
  Linear constraint matrix    : 0 Constrs, 0 Vars, 0 NZs
  Matrix coefficient range    : [ 0, 0 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 0, 0 ]


In [623]:
_, pmin, pmax = gp.multidict({
    k: [
        ppc['gen'][k, PMIN],
        ppc['gen'][k, PMAX]
    ] for k in range(Ng)
})

In [624]:
m.addConstrs((p.sum('*', t) == Demand.sum('*', t) for t in range(T)), name='balance')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [625]:
m.addConstrs(((p[k, t] - p[k, t - 1]) >= RD[k] for k in range(Ng) for t in range(1, T)), name='Ramp_lower')
m.addConstrs(((p[k, t] - p[k, t - 1]) <= RU[k] for k in range(Ng) for t in range(1, T)), name='Ramp_upper')
m.addConstrs(((y[k, t] + z[k, t]) <= 1 for k in range(Ng) for t in range(T)), name='start_shutdown_logic')
m.addConstrs((gp.quicksum([u[k, tau] for tau in range(t, t + TU[k])]) >= TU[k] * y[k, t] for k in range(Ng) for t in range(T - TU[k] + 1)), name='start_time')
m.addConstrs((gp.quicksum([1 - u[k, tau] for tau in range(t, t + TD[k])]) >= TD[k] * z[k, t] for k in range(Ng) for t in range(T - TD[k] + 1)), name='shutdown_time')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [626]:
from pandapower.pypower.idx_gen import GEN_BUS
PTDF = gp.tupledict({(l, b): ptdf[l][b] for l in range(Nl) for b in range(Nb)})
LODF = gp.tupledict({(l, c): lodf[l][c] for l in range(Nl) for c in range(Nl)})
Gen_bus = gp.tupledict({
    k: ppc['gen'][k, GEN_BUS] for k in range(Ng)
})
pl_max = gp.tupledict({
    l: ppc['branch'][l, RATE_A] for l in range(Nl)
})
m.addConstrs((gp.quicksum((PTDF[l, Gen_bus[k]] * p[k, t] for k in range(Ng))) - gp.quicksum((PTDF[l, b] * Demand[b, t] for b in range(Nb))) <= pl_max[l] for l in range(Nl) for t in range(T)), name='line_upper_base')
m.addConstrs((gp.quicksum((PTDF[l, Gen_bus[k]] * p[k, t] for k in range(Ng))) - gp.quicksum((PTDF[l, b] * Demand[b, t] for b in range(Nb))) >= -pl_max[l] for l in range(Nl) for t in range(T)), name='line_lower_base')
m.addConstrs((gp.quicksum((PTDF[l, Gen_bus[k]] * p[k, t] for k in range(Ng))) - gp.quicksum((PTDF[l, b] * Demand[b, t] for b in range(Nb))) <= pl_max[l] for l in range(Nl) for t in range(T)), name='line_upper_base')
m.addConstrs((gp.quicksum((PTDF[l, Gen_bus[k]] * p[k, t] for k in range(Ng))) - gp.quicksum((PTDF[l, b] * Demand[b, t] for b in range(Nb))) >= -pl_max[l] for l in range(Nl) for t in range(T)), name='line_lower_base')
for c in range(Nl):
    ok = True
    for ll in range(Nl):
        if np.isnan(LODF[ll, c]) or np.isinf(LODF[ll, c]):
            ok = False
            break
    if not ok:
        continue
    m.addConstrs((gp.quicksum(((PTDF[l, Gen_bus[k]] + LODF[l, c] * PTDF[c, Gen_bus[k]]) * p[k, t] for k in range(Ng))) - gp.quicksum(((PTDF[l, b] + LODF[l, c] * PTDF[c, b]) * Demand[b, t] for b in range(Nb))) <= pl_max[l] for l in range(Nl) for t in range(T)), name='line_upper_contingency')
    m.addConstrs((gp.quicksum(((PTDF[l, Gen_bus[k]] + LODF[l, c] * PTDF[c, Gen_bus[k]]) * p[k, t] for k in range(Ng))) - gp.quicksum(((PTDF[l, b] + LODF[l, c] * PTDF[c, b]) * Demand[b, t] for b in range(Nb))) >= -pl_max[l] for l in range(Nl) for t in range(T)), name='line_lower_contingency')

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


KeyboardInterrupt: 

In [ ]:
LODF[0, 9]

In [ ]:
m.optimize()

In [ ]:
m.getAttr('x', u)

In [ ]:
m.getAttr('x', p)

In [ ]:
a = m.getAttr('x', p)

In [ ]:
a.sum('*', '*')

In [ ]:
Demand.sum('*', '*')

In [ ]:
print(sum(ppc['gen'][:, PMAX]) * 24, sum(ppc['gen'][:, PMIN]) * 24)

In [ ]:
b = m.getAttr('x', u)
for k in range(Ng):
    for t in range(T):
        if b[k, t] == 0:
            print(k, t)

In [ ]:
for k in range(Ng):
    for t in range(T):
        if b[k, t] == 1:
            print(k, t)

In [ ]:
Cost_sample

In [ ]:
# m.computeIIS()